In [2]:
!nvidia-smi

Tue Sep 19 06:55:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [4]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline, AutoModelForCausalLM

**Data**

In [5]:
questions_dir = Path("skyscanner")
questions_dir.mkdir(exist_ok=True, parents=True)

def write_file(question, answer, file_path):
  text = f"""
  Q: {question}
  A: {answer}
  """.strip()
  with Path(questions_dir / file_path).open("w") as text_file:
    text_file.write(text)

In [6]:
write_file(
    question="What is Mitr Phol",
    answer="""Mitr Phol is Thailand's and Asia's biggest sugar and bio-energy producer.
    Mitr Phol Sugar Corp is a privately owned group of companies, mainly owned by the Vongkusolkit family.
    As of 2014, Mitr Phol is ranked as the world's fifth largest sugar producer,
     and the largest producer in Asia. It is Thailand's largest sugar producer and
     the second largest in China through its joint venture company East Asia Sugar.
     In addition to Thailand and China, Mitr Phol has operations and investments in Lao PDR, Cambodia, and
     most recently Australia. Its key business units include sugar, wood substitute materials, and renewable energy.
    """.strip(),
    file_path="question_1.txt",
)

In [7]:
write_file(
    question="Vision of Mitr Phol",
    answer="""To be the world-class sugar and bio-based leader
            by combining our fully integrated agribusiness model
            With innovative technology and management talent
            to creat value for a sustainable life for all.
    """.strip(),
    file_path="question_2.txt",
)

In [8]:
write_file(
    question="Mitr Phol Business",
    answer="""
    Mitr Phol have many business, namely, Sugarcane Development Business, Sugar Business,
    Energy Business, Wood Substitute Material, International Business, and Other Business.
    """.strip(),
    file_path="question_3.txt",
)

In [9]:
write_file(
    question="What is Mitr Phol's Sugarcane Development Business",
    answer="""The objective of Sugarcane Development Business is to do researching about sugarcane to support and developsugarcane gene. Make sustainability to farmer and sugarcane farm. Furthermore, the research will increase the productivity from 7 to 8 tons per rai to 20 tons per rai. Improving irrigation Setting up irrigation systems in and around sugarcane farms. Choosing the Right TechnologyTo be used in 376 villages in communities around Mitr Phol Group 6 sugar mills. Lastly, minimizing Costs.
    """.strip(),
    file_path="question_4.txt",
)

In [10]:
write_file(
    question="What is Mitr Phol's sugarcane business",
    answer="""
    Mitr Phol Group, the world 3rd largest sugar producer, is committed to business development. To add value to the sugarcane and sugar industries And the business has continued for more than half a century. Mitr Phol Group pays attention to every step of production. Give consumers confidence in world class quality standards.
    Make Mitr Phol well known and gain the confidence of customers. both domestically and abroad.
    """.strip(),
    file_path="question_5.txt",
)

In [11]:
write_file(
    question="How many Mitr Phol factories are there",
    answer="""
    Currently there are 7 factories consisting of Established in 1983,
Mitr Phu Khiao Sugar Factory Chaiyaphum Province,
production capacity 42,000 tons of sugarcane/day. Established in 1990,
Mitr Phol Sugar Factory, Dan Chang, Suphanburi Province,
production capacity 49,000 tons of sugar cane/day. Established in 1990,
Mitr Phu Wiang Sugar Factory Khon Kaen Province,
production capacity 36,000 tons of sugarcane/day. Established in 1997,
Singburi Sugar Factory Singburi Province,
production capacity 15,000 tons of sugarcane/day. Established in 1997,
Mitr Kalasin Sugar Factory Kalasin Province,
production capacity 39,000 tons of sugar cane/day. Established in 2012,
Mitr Phu Luang Sugar Factory, Loei Province,
production capacity 40,000 tons of sugar cane/day. Established in 2019,
Amnat Charoen Mitr Sugar Factory Amnat Charoen Province,
production capacity 15,000 tons of sugarcane/day
    """.strip(),
    file_path="question_6.txt",
)

In [12]:
write_file(
    question="What is Mitr Phol's energy business",
    answer="""
    Mitr Phol want to be a zero waste company in a short period of time, so we decided to create Bio power brightening the world with clean energy.
    Ethanol a clean alternative that is environmentally friendly and sell them out to the Electricity Authority. Also, bring back
    the steam power and power the machine.
    """.strip(),
    file_path="question_7.txt",
)

In [13]:
write_file(
    question="What is Mitr Phol's Wood Substitute Material Business",
    answer="""
    Panel Plus formerly MP Particle Board was established in 1990 under Mitr Phol Group management. We are a leading company that produces particle boards, MDF (medium density fiberboard) and melamine faced-chip boards, all of which are substitute materials for wood. With state of the art technology and highly experienced management, we produce 300,000 cubic meters of particle boards every year, 300,000 cubic meters of MDF every year, and 23.5 million square meters of melamine faced chip board every year.
    We are recognized across Asia as a leader in the industry, serving the needs of both domestic and international customers.
    """.strip(),
    file_path="question_8.txt",
)

In [14]:
write_file(
    question="What is Mitr Phol's international business",
    answer="""
     Mitr Phol want to expand our production base and adding economic value to the world. Present, Mitr Phol have invest
     their sugar mill to increase the product in 3 countries are China, Laos, and Australia.
    """.strip(),
    file_path="question_9.txt",
)

In [15]:
write_file(
    question="What is Mitr Phol product",
    answer="""
    The major product of Mitr Phol are sugar, molasses, fertilizer, and Wood substitute material.
    """.strip(),
    file_path="question_10.txt",
)

In [16]:
write_file(
    question="How many employee are in the Mitr Phol company",
    answer="""
    In Mitr Phol company there are approximately 7,400 employees divided to male 5,500 persons and female 1,900 persons.
    Nevertheless, Mitr Phol Group has contributed to the community income by hiring people from the neighboring area where our factory is located. Our recruitment helps supports them with work, occupation promotion, leveraging their life quality, and improving good relationships with the community. Basic compensation is paid in compliance with the laws. Through Strong Community Driven by the Disabled Project , the Company also employs person with disabilities from the communities to facilitate them for having workplace near their home, generating income for the disabilities, enabling them to be self-independent. Our contribution includes the support in establishing the Disabled Club to extend the career opportunity for other person with disability within the same sub-district. Such employment will reduce the inequality,
    encouraging the self-esteem among the disabled to live in the society with happiness.
    """.strip(),
file_path="question_11.txt",
)

In [17]:
write_file(
    question="Innovation and Reseacrh center of Mitr Phol",
    answer="""
    Mitr Phol Group realizes the importance and economic potential of sugarcane,
    which is the starting raw material for the sugar industry and other related industries.
    We established the Mitr Phol Innovation & Research Center Company Limited in 1997 to drive research
    and technological development that will help achieve sustainable sugarcane production and prep the group for business expansion. As the group is cane and sugar business evolved into an energy and bio plastics business, we elevated our R&D department to the Mitr Phol Innovation & Research Center to support our mission to become a socially and environmentally responsible company, with a vision of creating world class technology and green industries. We are committed to becoming a value adder through new initiatives, world class technology and innovation, delivering products and services that surpass the expectations of all of our stakeholders,
     and using cutting-edge knowledge to pave way for a truly sustainable business.
    """.strip(),
    file_path="question_12.txt",
)

In [18]:
write_file(
    question="Cane and Crop production",
    answer="""
    Our research & development efforts aim to increase the efficiency of modern cane farming management.
    Areas of research include alternative plants and biological pest control to reduce the usage of chemicals in cane farming.
    A team of experts help to distribute the knowledge to farmers.
    """.strip(),
    file_path="question_13.txt",
)

# Model

In [19]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [20]:
DEVICE

'cuda:0'

In [21]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)
generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [22]:
# model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
# model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
# model = AutoGPTQForCausalLM.from_quantized(
#         model_name_or_path,
#         model_basename=model_basename,
#         use_safetensors=True,
#         trust_remote_code=True,
#         device=DEVICE,
#     )
# generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [23]:
question = (
    "which programming language is more suitable for a beginner: Python or Javascript?"
)
prompt = f"""
###Instruction: {question}
###Response:
""".strip()

In [24]:
print(prompt)

###Instruction: which programming language is more suitable for a beginner: Python or Javascript?
###Response:


In [25]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(inputs = input_ids, temperature=0.7,max_new_tokens=512)

CPU times: user 3.76 s, sys: 615 ms, total: 4.38 s
Wall time: 7.09 s


In [26]:
print(tokenizer.decode(output[0]))

<s> ###Instruction: which programming language is more suitable for a beginner: Python or Javascript?
###Response:Python is generally considered more suitable for beginners due to its readability and simplicity compared to Javascript.</s>


In [27]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [28]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)

In [29]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Peg

In [30]:
llm = HuggingFacePipeline(pipeline=pipe)

In [31]:
response = llm(prompt)

Python is generally considered to be more suitable for beginners due to its readability and simplicity compared to JavaScript.


# **Embed Documents**

In [32]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs={"device": DEVICE},
)

In [33]:
loader = DirectoryLoader("./skyscanner/", glob="**/*txt")
documents = loader.load()
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


13

In [34]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [35]:
texts[4]

Document(page_content='Q: How many employee are in the Mitr Phol company A: In Mitr Phol company there are approximately 7,400 employees divided to male 5,500 persons and female 1,900 persons. Nevertheless, Mitr Phol Group has contributed to the community income by hiring people from the neighboring area where our factory is located. Our recruitment helps supports them with work, occupation promotion, leveraging their life quality, and improving good relationships with the community. Basic compensation is paid in compliance with the laws. Through Strong Community Driven by the Disabled Project , the Company also employs person with disabilities from the communities to facilitate them for having workplace near their home, generating income for the disabilities, enabling them to be self-independent. Our contribution includes the support in establishing the Disabled Club to extend the career opportunity for other person with disability within the same sub-district. Such employment will re

In [36]:
db = Chroma.from_documents(texts, embeddings)

In [37]:
db.similarity_search("flight search")

[Document(page_content='Q: Innovation and Reseacrh center of Mitr Phol A: Mitr Phol Group realizes the importance and economic potential of sugarcane, which is the starting raw material for the sugar industry and other related industries. We established the Mitr Phol Innovation & Research Center Company Limited in 1997 to drive research and technological development that will help achieve sustainable sugarcane production and prep the group for business expansion. As the group is cane and sugar business evolved into an energy and bio plastics business, we elevated our R&D department to the Mitr Phol Innovation & Research Center to support our mission to become a socially and environmentally responsible company, with a vision of creating world class technology and green industries. We are committed to becoming a value adder through new initiatives, world class technology and innovation, delivering products and services that surpass the expectations of all of our stakeholders, and using c

# Conversational chain

In [38]:
template = """
### Instruction: You're a advisor of a company that need to give the company information to customers. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. IF you don't know the answer ~ say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### input: {question}
### Response:
""".strip()

In [39]:
prompt = PromptTemplate(
    input_variables={"context", "question", "chat_history"}, template=template
)

In [40]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="answer",
    return_messages=True,
)

In [41]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True,
)

In [42]:
question = "How flight search works?"
answer = chain(question)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a advisor of a company that need to give the company information to customers. Use only the chat history and the following information
Q: Vision of Mitr Phol A: To be the world-class sugar and bio-based leader by combining our fully integrated agribusiness model With innovative technology and management talent to creat value for a sustainable life for all.

Q: Innovation and Reseacrh center of Mitr Phol A: Mitr Phol Group realizes the importance and economic potential of sugarcane, which is the starting raw material for the sugar industry and other related industries. We established the Mitr Phol Innovation & Research Center Company Limited in 1997 to drive research and technological development that will help achieve sustainable sugarcane production and prep the group for business expansion. As the group is cane and sugar business evolved into an energy and bio plastics business, we 

In [43]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [44]:
answer["source_documents"]

[Document(page_content='Q: Vision of Mitr Phol A: To be the world-class sugar and bio-based leader by combining our fully integrated agribusiness model With innovative technology and management talent to creat value for a sustainable life for all.', metadata={'source': 'skyscanner/question_2.txt'}),
 Document(page_content='Q: Innovation and Reseacrh center of Mitr Phol A: Mitr Phol Group realizes the importance and economic potential of sugarcane, which is the starting raw material for the sugar industry and other related industries. We established the Mitr Phol Innovation & Research Center Company Limited in 1997 to drive research and technological development that will help achieve sustainable sugarcane production and prep the group for business expansion. As the group is cane and sugar business evolved into an energy and bio plastics business, we elevated our R&D department to the Mitr Phol Innovation & Research Center to support our mission to become a socially and environmentally 

In [45]:
question = "I bought flight tickets, but I can't find any confirmation. Where is it?"
response = chain(question)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How flight search works?
Assistant: Flight search engines work by scanning multiple airlines and travel websites to find available flights based on specific criteria such as departure date, arrival destination, number of passengers, and preferred dates/times. They then display a list of options ranked by price or relevance, allowing users to compare prices and choose the best option for their needs.
Follow Up Input: I bought flight tickets, but I can't find any confirmation. Where is it?
Standalone question:
Where is my flight ticket confirmation?

> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a advisor of a company that need to give the company information to customers. Use only the chat histor

# QA Chain with Memory

In [46]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="output_text",
    return_messages=False,
)
chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

# Support Chatbot

In [47]:
DEFAULT_TEMPLATE = """
### Instruction: You're a advisor of a company that need to give the company information to customers. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. IF you don't know the answer ~ say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### input: {question}
### Response:
""".strip()

class Chatbot:
  def __init__(
      self,
      text_pipeline: HuggingFacePipeline,
      embeddings: HuggingFaceEmbeddings,
      documents_dir: Path,
      prompt_template: str = DEFAULT_TEMPLATE,
      verbose: bool=False,
  ):
      prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template=prompt_template,
      )
      self.chain = self._create_chain(text_pipeline, prompt, verbose)
      self.db = self._embed_data(documents_dir, embeddings)

  def _create_chain(
      self,
      text_pipeline: HuggingFacePipeline,
      prompt: PromptTemplate,
      verbose: bool = False,
  ):
      memory = ConversationBufferMemory(
          memory_key="chat_history",
          human_prefix="### Input",
          ai_prefix="### Response",
          input_key="question",
          output_key="output_text",
          return_messages=False,
      )
      return load_qa_chain(
          text_pipeline,
          chain_type="stuff",
          prompt=prompt,
          memory=memory,
          verbose=verbose,
      )

  def _embed_data(
      self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
  )-> Chroma:
      loader = DirectoryLoader(documents_dir, glob="**/*txt")
      documents = loader.load()
      text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
      texts = text_splitter.split_documents(documents)
      return Chroma.from_documents(texts, embeddings)
  def __call__(self, user_input: str)->str:
    docs = self.db.similarity_search(user_input)
    return self.chain.run({"input_documents": docs, "question": user_input})

In [48]:
chatbot = Chatbot(llm, embeddings, "./skyscanner/")

In [ ]:
# model.save_pretrained('./model')
# tokenizer.save_pretrained('./model')

In [51]:
# model = Automodel.from_pretrained('./model')

In [50]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

while True:
  user_input = input("Type bye to exit\nYour prompt: ")
  if user_input.lower() in ["bye", "goodbye"]:
    break
  answer = chatbot(user_input)
  print()

Type bye to exit
Your prompt: bie the ska is billy father?
I'm sorry but I do not have enough information about Billy or his father to provide an accurate response. Could you please provide more details or context about what you would like me to find out?

Type bye to exit
Your prompt: bie the ska is billy father
I apologize for any confusion earlier. Based on my understanding, "bie the ska" is likely a misspelling of "beet the skate," which refers to a type of dance move. However, without additional context or clarification, it's difficult for me to determine if there's a connection to Billy or his father.

Type bye to exit
Your prompt: is bie the ska is Billy's father
As mentioned previously, without more information or context it's challenging for me to accurately respond to your inquiry. If you could provide more details or clarify your question, I'll be happy to assist you further.

Type bye to exit
Your prompt: tong bid?
Tong Bid is a Filipino term that translates to "thank you."